In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '../data_misc/eia_form923/**/EIA923_Schedule_8_Annual_Environmental_Information_*_Final_Revision.xlsx'
p2 = '../data_misc/eia_form923/**/EIA923_Schedule_8_Annual_Environmental_Information_**_Final.xlsx'
files = glob.glob(path)
files = files + glob.glob(p2)
files.append('../data_misc/eia_form923/f923_2017/EIA923_Schedule_8_Annual_Envir_Infor_2017_Final.xlsx')
files.append('../data_misc/eia_form923/f923_2013/EIA923_Schedule_8_PartsA-D_EnvData_2013_Final_Revision.xlsx')
files.sort()
files

['../data_misc/eia_form923/f923_2012/EIA923_Schedule_8_Annual_Environmental_Information_2012_Final_Revision.xlsx',
 '../data_misc/eia_form923/f923_2013/EIA923_Schedule_8_PartsA-D_EnvData_2013_Final_Revision.xlsx',
 '../data_misc/eia_form923/f923_2014/EIA923_Schedule_8_Annual_Environmental_Information_2014_Final_Revision.xlsx',
 '../data_misc/eia_form923/f923_2015/EIA923_Schedule_8_Annual_Environmental_Information_2015_Final_Revision.xlsx',
 '../data_misc/eia_form923/f923_2016/EIA923_Schedule_8_Annual_Environmental_Information_2016_Final_Revision.xlsx',
 '../data_misc/eia_form923/f923_2017/EIA923_Schedule_8_Annual_Envir_Infor_2017_Final.xlsx',
 '../data_misc/eia_form923/f923_2018/EIA923_Schedule_8_Annual_Environmental_Information_2018_Final.xlsx',
 '../data_misc/eia_form923/f923_2019/EIA923_Schedule_8_Annual_Environmental_Information_2019_Final_Revision.xlsx',
 '../data_misc/eia_form923/f923_2020/EIA923_Schedule_8_Annual_Environmental_Information_2020_Final_Revision.xlsx']

In [3]:
frames = []
# only include data from 2016 and beyond. Earlier datasets appear to have quality issues.
for f in files[4:]:
    df = pd.read_excel(f, 
                       skiprows=4, 
                       sheet_name='8D Cooling System Information',
                       usecols=[0,1,2,8,9,10,11,13,14,15,16],
                       na_filter=True,
                       na_values='.')    
    frames.append(df)
full_df = pd.concat(frames, axis=0)

In [4]:
full_df

,Year,Month,Plant ID,Diversion Rate \n(gallons per minute),Withdrawal Rate \n(gallons per minute),Discharge Rate \n(gallons per minute),Consumption Rate \n(gallons per minute),Intake Average\nTemperature (ºF),Intake Maximum\nTemperature (ºF),Discharge Average\nTemperature (ºF),Discharge Maximum\nTemperature (ºF)
0,2016,1,3,NaN,90243.0,90243.0,0.0,53.0,62.0,62.0,71.0
1,2016,1,3,NaN,397236.0,395444.0,1792.0,53.0,62.0,62.0,71.0
2,2016,1,3,NaN,NaN,NaN,1535.0,NaN,NaN,NaN,NaN
3,2016,1,3,NaN,NaN,NaN,1458.0,NaN,NaN,NaN,NaN
4,2016,2,3,NaN,144285.0,144285.0,0.0,53.0,57.0,62.0,74.0
...,...,...,...,...,...,...,...,...,...,...,...
15445,2020,8,64020,0.0,1802.1,150.2,1651.9,NaN,NaN,76.0,85.0
15446,2020,9,64020,0.0,1334.9,221.7,1113.2,NaN,NaN,73.0,79.0
15447,2020,10,64020,0.0,1643.3,244.6,1398.7,NaN,NaN,72.0,79.0
15448,2020,11,64020,0.0,710.9,141.7,569.2,NaN,NaN,71.0,78.0


In [5]:
ot_plants = pd.read_csv('nuclear_plants_OT.csv')
codes = np.array(ot_plants['Plant Code'].values)
names = np.array(ot_plants['Plant Name'].values)

In [6]:
ot_plants.rename(columns={'Plant Code':'Plant ID'},inplace=True)
ot_plants['Plant ID'] = ot_plants['Plant ID'].apply(int)

In [7]:
full_df = full_df.merge(ot_plants, on='Plant ID', how='left')

In [8]:
is_nuclear_plant = lambda x: x in codes
full_df['is_npp'] = full_df['Plant ID'].apply(is_nuclear_plant)
nuclear_df = full_df[full_df['is_npp'] == True]

In [9]:
nuclear_df

,Year,Month,Plant ID,Diversion Rate \n(gallons per minute),Withdrawal Rate \n(gallons per minute),Discharge Rate \n(gallons per minute),Consumption Rate \n(gallons per minute),Intake Average\nTemperature (ºF),Intake Maximum\nTemperature (ºF),Discharge Average\nTemperature (ºF),Discharge Maximum\nTemperature (ºF),Nameplate Capacity (MW),Plant Name,State,max_withdrawal_rate_kgM,is_npp
168,2016,1,46,NaN,2113996.0,2113611.0,385.0,50.0,59.0,59.0,59.0,3494.0,Browns Ferry,AL,1979794.29,True
169,2016,2,46,NaN,2118280.0,2113611.0,4669.0,49.0,51.0,53.0,55.0,3494.0,Browns Ferry,AL,1979794.29,True
170,2016,3,46,NaN,2118758.0,2113611.0,5147.0,60.0,62.0,68.0,71.0,3494.0,Browns Ferry,AL,1979794.29,True
171,2016,4,46,NaN,1962803.0,1962153.0,650.0,71.0,72.0,79.0,81.0,3494.0,Browns Ferry,AL,1979794.29,True
172,2016,5,46,NaN,1978493.0,1977917.0,576.0,80.0,82.0,80.0,87.0,3494.0,Browns Ferry,AL,1979794.29,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75006,2020,8,8907,0.0,36732.0,36732.0,0.0,79.0,81.0,100.0,105.0,1012.0,Indian Point 3,NY,840211.95,True
75007,2020,9,8907,0.0,35830.0,35830.0,0.0,75.0,78.0,94.0,100.0,1012.0,Indian Point 3,NY,840211.95,True
75008,2020,10,8907,0.0,36371.0,36371.0,0.0,68.0,71.0,77.0,91.0,1012.0,Indian Point 3,NY,840211.95,True
75009,2020,11,8907,0.0,32651.0,32651.0,0.0,59.0,64.0,83.0,87.0,1012.0,Indian Point 3,NY,840211.95,True


In [10]:
npp_groups = nuclear_df.groupby('Plant ID')

In [11]:
to_celcius = lambda T: 5/9*(T-32)

In [12]:
column_labels= {'Diversion Rate \n(gallons per minute)':'diversion_rate_kgM',
                'Withdrawal Rate \n(gallons per minute)':'withdrawal_rate_kgM',
                'Discharge Rate \n(gallons per minute)':'discharge_rate_kgM',
                'Intake Average\nTemperature (ºF)':'avg_intake_temp_C', 
                'Intake Maximum\nTemperature (ºF)':'max_intake_temp_C',
                'Discharge Average\nTemperature (ºF)':'avg_discharge_temp_C',
                'Discharge Maximum\nTemperature (ºF)':'max_discharge_temp_C',
                'Consumption Rate \n(gallons per minute)':'consumption_rate_kgM'}

In [13]:
for i, g in enumerate(npp_groups.groups):
    plant = names[i].replace(' ',"").lower()
    # get the plant specific data
    df = npp_groups.get_group(g)
    # create a day column for a datetime index
    df['Day'] = 1
    # create the datetime column
    cols = ['Year', 'Month', 'Day']
    df['date'] = pd.to_datetime(df[cols])
    df.drop(columns=cols, inplace=True)
    # create sensible column names
    df.rename(columns=column_labels, inplace=True)
    df.set_index('date', inplace=True)
    # replace NaN with 0
    df.fillna(0,inplace=True)
    # convert gallons per minute to kg per month
    df['DaysPerMonth'] = df.index.daysinmonth
    df.iloc[:,1:5] = df.iloc[:,1:5].multiply(df['DaysPerMonth'],axis='index')*3.76*24*60
    df['max_withdrawal_rate_kgM'] = df['max_withdrawal_rate_kgM'].multiply(df['DaysPerMonth'],
                                                                           axis='index')*3.76*24*60
    # convert fahrenheit to celcius
    df.iloc[:,5:8] = df.iloc[:,5:8].apply(to_celcius)
    df.drop(columns=['DaysPerMonth', 'is_npp'], inplace=True)
    df.to_csv(f'cooling_data/{plant}_cooling.csv')
    print(plant, f" saved to cooling_data/{plant}_cooling.csv")

brownsferry  saved to cooling_data/brownsferry_cooling.csv
clintonpowerstation  saved to cooling_data/clintonpowerstation_cooling.csv
wolfcreekgeneratingstation  saved to cooling_data/wolfcreekgeneratingstation_cooling.csv
millstone  saved to cooling_data/millstone_cooling.csv
turkeypoint  saved to cooling_data/turkeypoint_cooling.csv
dresdengeneratingstation  saved to cooling_data/dresdengeneratingstation_cooling.csv
quadcitiesgeneratingstation  saved to cooling_data/quadcitiesgeneratingstation_cooling.csv
monticellonuclearfacility  saved to cooling_data/monticellonuclearfacility_cooling.csv
prairieisland  saved to cooling_data/prairieisland_cooling.csv
psegsalemgeneratingstation  saved to cooling_data/psegsalemgeneratingstation_cooling.csv
ninemilepointnuclearstation  saved to cooling_data/ninemilepointnuclearstation_cooling.csv
peachbottom  saved to cooling_data/peachbottom_cooling.csv
hbrobinson  saved to cooling_data/hbrobinson_cooling.csv
oconee  saved to cooling_data/oconee_cool

In [14]:
df

,Plant ID,diversion_rate_kgM,withdrawal_rate_kgM,discharge_rate_kgM,consumption_rate_kgM,avg_intake_temp_C,max_intake_temp_C,avg_discharge_temp_C,max_discharge_temp_C,Nameplate Capacity (MW),Plant Name,State,max_withdrawal_rate_kgM
date,,,,,,,,,,,,,
2016-01-01,8907,0.0,8.433258e+10,8.433258e+10,0.0,2.222222,4.444444,17.222222,67.0,1012.0,Indian Point 3,NY,1.410266e+11
2016-02-01,8907,0.0,7.355992e+10,7.355992e+10,0.0,1.111111,2.777778,14.444444,62.0,1012.0,Indian Point 3,NY,1.319281e+11
2016-03-01,8907,0.0,8.382753e+10,8.382753e+10,0.0,1.666667,2.222222,12.222222,58.0,1012.0,Indian Point 3,NY,1.410266e+11
2016-04-01,8907,0.0,6.934888e+10,6.934888e+10,0.0,7.222222,7.777778,20.555556,71.0,1012.0,Indian Point 3,NY,1.364773e+11
2016-05-01,8907,0.0,5.851528e+10,5.851528e+10,0.0,16.111111,16.666667,26.111111,82.0,1012.0,Indian Point 3,NY,1.410266e+11
2016-06-01,8907,0.0,1.294526e+11,1.294526e+11,0.0,21.666667,22.222222,32.222222,92.0,1012.0,Indian Point 3,NY,1.364773e+11
2016-07-01,8907,0.0,1.390139e+11,1.390139e+11,0.0,24.444444,25.000000,35.000000,98.0,1012.0,Indian Point 3,NY,1.410266e+11
2016-08-01,8907,0.0,1.387250e+11,1.387250e+11,0.0,26.666667,27.777778,37.777778,102.0,1012.0,Indian Point 3,NY,1.410266e+11
2016-09-01,8907,0.0,9.945841e+10,9.945841e+10,0.0,26.111111,26.666667,36.111111,99.0,1012.0,Indian Point 3,NY,1.364773e+11
